<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

### **Transformación de tipos complejos**

Consultar datos tabulares almacenados en el Data Lakehouse con Spark SQL es fácil, eficiente y rápido.

Esto se complica a medida que la estructura de datos se vuelve menos regular, cuando es necesario utilizar muchas tablas en una sola consulta o cuando es necesario cambiar drásticamente la forma de los datos. Este notebook introduce una serie de funciones presentes en Spark SQL para ayudar a los ingenieros a completar incluso las transformaciones más complicadas.

#### **Objetivos de aprendizaje**
Al final de esta lección, deberías ser capaz de:
- Utilizar la sintaxis **`.`** y **`:`** para consultar datos anidados
- Parsear JSON strings en structs
- Aplanar y desempaquetar arrays y structs
- Combinar datasets mediante joins
- Reformar datos mediante tablas pivot

#### **Run Setup**

El setup script creará los datos y declarará los valores necesarios para que el resto de este notebook se ejecute.

In [ ]:
%run ../Includes/Classroom-Setup-02.5

#### **Visión general de los datos**

La tabla **`events_raw`** se ha registrado con datos que representan una carga útil de Kafka. En la mayoría de los casos, los datos de Kafka serán valores JSON con codificación binaria. 

Convirtamos **`key`** y **`value`** en strings para ver estos valores en un formato legible por humanos.

In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW events_strings AS 
SELECT string(key), string(value) 
FROM events_raw;

SELECT * FROM events_strings

<center><img src="https://i.postimg.cc/MT4ZTL1w/db374.png"></center>

In [ ]:
%python
from pyspark.sql.functions import col

events_stringsDF = (spark
    .table("events_raw")
    .select(col("key").cast("string"), 
            col("value").cast("string"))
    )
display(events_stringsDF)

<center><img src="https://i.postimg.cc/MT4ZTL1w/db374.png"></center>

#### **Trabajar con datos anidados**

La siguiente celda de código consulta las cadenas convertidas para ver un objeto JSON de ejemplo sin campos nulos (lo necesitaremos para la siguiente sección).

**NOTA:** Spark SQL tiene funcionalidad incorporada para interactuar directamente con datos anidados almacenados como JSON strings o struct types.
- Utiliza la sintaxis **`:`** en las consultas para acceder a los subcampos de las cadenas JSON.
- Utiliza la sintaxis **`.`** en las consultas para acceder a los subcampos de los tipos struct.

In [ ]:
%sql
SELECT * FROM events_strings 
WHERE value:event_name = "finalize" 
ORDER BY key LIMIT 1

<center><img src="https://i.postimg.cc/C1TzKMXF/db375.png"></center>

In [ ]:
%python
display(events_stringsDF
    .where("value:event_name = 'finalize'")
    .orderBy("key")
    .limit(1)
)

<center><img src="https://i.postimg.cc/pXGrFJ4c/db376.png"></center>

Utilicemos el ejemplo del JSON string anterior para derivar el schema, y luego analicemos toda la columna JSON en tipos struct.
- **`schema_of_json()`** devuelve el schema derivado de una cadena JSON de ejemplo.
- **`from_json()`** analiza una columna que contiene una cadena JSON en un tipo struct utilizando el schema especificado.

Después de descomprimir la cadena JSON en un tipo struct, vamos a descomprimir y aplanar todos los campos struct en columnas.
- **`*`** unpacking puede ser usado para aplanar structs; **`col_name.*`** extrae los subcampos de **`col_name`** en sus propias columnas.

In [ ]:
%sql
SELECT schema_of_json('{"device":"Linux",
                       "ecommerce":{"purchase_revenue_in_usd":1075.5,"total_item_quantity":1,"unique_items":1},
                       "event_name":"finalize",
                       "event_previous_timestamp":1593879231210816,
                       "event_timestamp":1593879335779563,
                       "geo":{"city":"Houston","state":"TX"},
                       "items":[{"coupon":"NEWBED10",
                                 "item_id":"M_STAN_K",
                                 "item_name":"Standard King Mattress",
                                 "item_revenue_in_usd":1075.5,
                                 "price_in_usd":1195.0,
                                 "quantity":1}],
                       "traffic_source":"email",
                       "user_first_touch_timestamp":1593454417513109,
                       "user_id":"UA000000106116176"}') AS schema

<center><img src="https://i.postimg.cc/wvX3R8YD/db378.png"></center>

In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW parsed_events 
AS SELECT json.* FROM (
                        SELECT from_json(value, 'STRUCT<device: STRING, ecommerce: STRUCT<purchase_revenue_in_usd: DOUBLE, total_item_quantity: BIGINT, unique_items: BIGINT>, event_name: STRING, event_previous_timestamp: BIGINT, event_timestamp: BIGINT, geo: STRUCT<city: STRING, state: STRING>, items: ARRAY<STRUCT<coupon: STRING, item_id: STRING, item_name: STRING, item_revenue_in_usd: DOUBLE, price_in_usd: DOUBLE, quantity: BIGINT>>, traffic_source: STRING, user_first_touch_timestamp: BIGINT, user_id: STRING>') AS json 
                        FROM events_strings);

SELECT * FROM parsed_events

In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW parsed_events 
AS SELECT json.* FROM (
                        SELECT from_json(value, schema_of_json('{"device":"Linux","ecommerce":{"purchase_revenue_in_usd":1075.5,"total_item_quantity":1,"unique_items":1},"event_name":"finalize","event_previous_timestamp":1593879231210816,"event_timestamp":1593879335779563,"geo":{"city":"Houston","state":"TX"},"items":[{"coupon":"NEWBED10","item_id":"M_STAN_K","item_name":"Standard King Mattress","item_revenue_in_usd":1075.5,"price_in_usd":1195.0,"quantity":1}],"traffic_source":"email","user_first_touch_timestamp":1593454417513109,"user_id":"UA000000106116176"}')) AS json 
                        FROM events_strings);

SELECT * FROM parsed_events

<center><img src="https://i.postimg.cc/3xzwLJKg/db377.png"></center>

In [ ]:
%python
from pyspark.sql.functions import from_json, schema_of_json

json_string = """
{"device":"Linux","ecommerce":{"purchase_revenue_in_usd":1047.6,"total_item_quantity":2,"unique_items":2},"event_name":"finalize","event_previous_timestamp":1593879787820475,"event_timestamp":1593879948830076,"geo":{"city":"Huntington Park","state":"CA"},"items":[{"coupon":"NEWBED10","item_id":"M_STAN_Q","item_name":"Standard Queen Mattress","item_revenue_in_usd":940.5,"price_in_usd":1045.0,"quantity":1},{"coupon":"NEWBED10","item_id":"P_DOWN_S","item_name":"Standard Down Pillow","item_revenue_in_usd":107.10000000000001,"price_in_usd":119.0,"quantity":1}],"traffic_source":"email","user_first_touch_timestamp":1593583891412316,"user_id":"UA000000106459577"}
"""
parsed_eventsDF = (events_stringsDF
    .select(from_json("value", schema_of_json(json_string)).alias("json"))
    .select("json.*")
)

display(parsed_eventsDF)

<center><img src="https://i.postimg.cc/rwDmCtLV/db379.png"></center>

#### **Manipular Arrays**

Spark SQL tiene una serie de funciones para manipular array data, incluyendo las siguientes:
- **`explode()`** separa los elementos de un array en múltiples filas; esto crea una nueva fila para cada elemento.
- **`size()`** proporciona un recuento del número de elementos de un array para cada fila.

El código siguiente descompone (explodes) el campo **`items`** (un array de structs) en múltiples filas y muestra los eventos que contienen arrays con 3 o más elementos.

In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW exploded_events AS
SELECT *, explode(items) AS item
FROM parsed_events;

SELECT * FROM exploded_events WHERE size(items) > 2

<center><img src="https://i.postimg.cc/zvqvntq6/db380.png"></center>
<center><img src="https://i.postimg.cc/5tv2Dd0D/db381.png"></center>
<center><img src="https://i.postimg.cc/3N9R9XRR/db382.png"></center>
<center><img src="https://i.postimg.cc/FFx794Z4/db383.png"></center>

In [ ]:
%python
from pyspark.sql.functions import explode, size

exploded_eventsDF = (parsed_eventsDF
    .withColumn("item", explode("items"))
)

display(exploded_eventsDF.where(size("items") > 2))

<center><img src="https://i.postimg.cc/Pxj5j3vJ/db384.png"></center>

In [ ]:
%sql
DESCRIBE exploded_events

El siguiente código combina transformaciones de arrays para crear una tabla que muestra la colección única de actions y los items en el carrito de un usuario.
- **`collect_set()`** recoge valores únicos para un campo, incluyendo campos dentro de arrays.
- **`flatten()`** combina múltiples arrays en un único array.
- **`array_distinct()`** elimina los elementos duplicados de un array.

In [ ]:
%sql
SELECT user_id,
  collect_set(event_name) AS event_history,
  array_distinct(flatten(collect_set(items.item_id))) AS cart_history
FROM exploded_events
GROUP BY user_id

<center><img src="https://i.postimg.cc/fLfTTcYK/db385.png"></center>

In [ ]:
%python

from pyspark.sql.functions import array_distinct, collect_set, flatten

display(exploded_eventsDF
    .groupby("user_id")
    .agg(collect_set("event_name").alias("event_history"),
            array_distinct(flatten(collect_set("items.item_id"))).alias("cart_history"))
)

<center><img src="https://i.postimg.cc/90XX4Dpj/db386.png"></center>

#### **Join Tables**

Spark SQL soporta operaciones estándar **`JOIN`** (inner, outer, left, right, anti, cross, semi).  
Aquí joineamos el dataset de eventos explotados (exploded events dataset) con una tabla lookup para obtener el nombre estándar del ítem impreso.

In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW item_purchases AS

SELECT * 
FROM (SELECT *, explode(items) AS item FROM sales) a
INNER JOIN item_lookup b
ON a.item.item_id = b.item_id;

SELECT * FROM item_purchases

<center><img src="https://i.postimg.cc/T1kdnT75/db387.png"></center>
<center><img src="https://i.postimg.cc/2jT8sxkj/db388.png"></center>
<center><img src="https://i.postimg.cc/KvzvxgjP/db389.png"></center>
<center><img src="https://i.postimg.cc/SNvQsNYw/db390.png"></center>

In [ ]:
%python
exploded_salesDF = (spark
    .table("sales")
    .withColumn("item", explode("items"))
)

itemsDF = spark.table("item_lookup")

item_purchasesDF = (exploded_salesDF
    .join(itemsDF, exploded_salesDF.item.item_id == itemsDF.item_id)
)

display(item_purchasesDF)

<center><img src="https://i.postimg.cc/W1qNJLMt/db391.png"></center>

#### **Tablas Pivot**

Podemos utilizar **`PIVOT`** para ver los datos desde diferentes perspectivas mediante la rotación de valores únicos en una columna pivote especificada en múltiples columnas basadas en una función agregada.
- La cláusula **`PIVOT`** sigue al nombre de la tabla o subconsulta especificada en una cláusula **`FROM`**, que es la entrada para la tabla pivot.
- Los valores únicos de la columna pivotante se agrupan y agregan utilizando la expresión de agregación proporcionada, creando una columna independiente para cada valor único en la tabla pivotante resultante.

La siguiente celda de código utiliza **`PIVOT`** para aplanar la información de compra de ítems contenida en varios campos derivados del dataset **`sales`**. Este formato de datos aplanado puede ser útil para la creación de dashboards, pero también para aplicar algoritmos de machine learning para inferencia o predicción.

In [ ]:
%sql
SELECT *
FROM item_purchases
PIVOT (
  sum(item.quantity) FOR item_id IN (
    'P_FOAM_K',
    'M_STAN_Q',
    'P_FOAM_S',
    'M_PREM_Q',
    'M_STAN_F',
    'M_STAN_T',
    'M_PREM_K',
    'M_PREM_F',
    'M_STAN_K',
    'M_PREM_T',
    'P_DOWN_S',
    'P_DOWN_K')
)

<center><img src="https://i.postimg.cc/6Qs9rj0c/db392.png"></center>
<center><img src="https://i.postimg.cc/1tBsZ9Z7/db395.png"></center>
<center><img src="https://i.postimg.cc/V6mmTJ2b/db394.png"></center>
<center><img src="https://i.postimg.cc/s2p3wYHr/db393.png"></center>

In [ ]:
%python
transactionsDF = (item_purchasesDF
    .groupBy("order_id", 
        "email",
        "transaction_timestamp", 
        "total_item_quantity", 
        "purchase_revenue_in_usd", 
        "unique_items",
        "items",
        "item",
        "name",
        "price")
    .pivot("item_id")
    .sum("item.quantity")
)
display(transactionsDF)

Ejecute la siguiente celda para eliminar las tablas y archivos asociados a esta lección.

In [ ]:
%python
DA.cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>